In [1]:
import pandas as pd
import json

# load portuguese legal QnA datasetL LegalQA
dataset = pd.read_csv("/kaggle/input/balanced-5-8k/balanced_AI_5.8k.csv")

def buildprompt(data):
    prompt = {}
    prompt['text'] = "Given the question delimited by triple backticks  ```{" + data['question'] + "}```, what is the answer?  Answer: {" + data['answer'] + "}"    
    return prompt

dataset['prompt'] = dataset.apply(buildprompt, axis=1)

result = dataset['prompt'].to_list()
# save prompts to a json file
with open('prompts.json', 'w') as outfile:
    json.dump(result, outfile, ensure_ascii=False)

In [2]:
import json

# Open and read the JSON file
with open('/kaggle/working/prompts.json', 'r') as infile:
    prompts_data = json.load(infile)

# Display the contents of the JSON file
print(prompts_data[0])

{'text': 'Given the question delimited by triple backticks  ```{مریم آپ ان بلز کی کاپی کروا لینا یاد سے}```, what is the answer?  Answer: {ہاں}'}


In [3]:
import pandas as pd
import torch
import json
from transformers import BloomTokenizerFast, BloomForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
from accelerate import Accelerator

2024-05-16 07:48:33.544238: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 07:48:33.544343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 07:48:33.672452: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:

# Loading bloomz model and tokenizer 
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloomz-1b1")
model = BloomForCausalLM.from_pretrained("bigscience/bloomz-1b1")

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

In [5]:
dataset = load_dataset("json", data_files="prompts.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 5830
    })
})

In [7]:
# prepare the data for training
def prepare_train_data(data):
    text_input = data['text']
    tokenized_input = tokenizer(text_input, return_tensors='pt', padding=True, truncation=True, max_length=200)
    # Ensure labels are tensors of shape (batch_size, sequence_length)
    tokenized_input['labels'] = tokenized_input['input_ids'].clone()
    return tokenized_input


In [8]:
train_dataset = dataset['train'].map(prepare_train_data, batched=True, remove_columns=["text"], batch_size=128)

Map:   0%|          | 0/5830 [00:00<?, ? examples/s]

In [9]:
# Inspect the data
print("Length of input sequences:")
for i in range(5):  # Print information for the first 5 samples
    sample = train_dataset[i]
    print(len(sample['input_ids']))

Length of input sequences:
106
106
106
106
106


In [10]:
# Check tokenization
sample_text = "اچھا تو کیا اپ ڈیٹس ہیں آپ لوگوں کے پاس انٹرنز کے حوالے سے کیونکہ آپ کو یاد ہو گا کہ لاسٹ ٹائم میں نے کہا تھا کہ نیکسٹ میٹنگ میں ہم لوگ انٹرنز جو ہیں وہ فائنلائز کریں گے"
tokenized_text = tokenizer(sample_text)
print("Tokenized text:", tokenized_text)

Tokenized text: {'input_ids': [208519, 4264, 4946, 5870, 18488, 23923, 3415, 9374, 23671, 1202, 28703, 63471, 31011, 1202, 35559, 2045, 47049, 9374, 2174, 41585, 2945, 10401, 2379, 129285, 1466, 169321, 1468, 2563, 7977, 7235, 2379, 105694, 18848, 117515, 1468, 10195, 14277, 63471, 31011, 5353, 3415, 6560, 152169, 25229, 18867, 12473], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [11]:
# setting arguments to be used during training
training_arguments = TrainingArguments(
    'AI-bloom-1b1-5.8k',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    num_train_epochs=2,
    weight_decay=0.01,
    optim="adafactor",
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    save_strategy='no'
)

In [12]:
from transformers import DataCollatorForLanguageModeling


# # Configure the tokenizer for padding and truncation
# tokenizer.padding = True
# tokenizer.truncation = True

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Train on causal language modeling instead of masked language modeling
    pad_to_multiple_of=8  # Set the padding to a multiple of 8 (adjust as needed)
)

In [13]:
trainer = Trainer(
    model=model,
    args=training_arguments,
    data_collator=data_collator,  # Pass the data collator here
    train_dataset=train_dataset
)


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


In [ ]:
trainer.save_model("AI_Bloomz1b1_5.8k")

In [ ]:
import shutil
shutil.make_archive('finetuned_AI_Bloomz1b1_5.8k', 'zip', '/kaggle/working/AI_Bloomz1b1_5.8k')

In [ ]:
from IPython.display import FileLink
FileLink(r'finetuned_AI_Bloomz1b1_5.8k.zip')

In [ ]:
import csv
import torch
import time
from transformers import pipeline, BloomTokenizerFast, BloomForCausalLM

# Loading the fine-tuned model: LegalQA-bloom-560m
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloomz-1b1")
model = BloomForCausalLM.from_pretrained("/kaggle/working/AI_Bloomz1b1_5.8k")
prompt = 'Given the question delimited by triple backticks ```{{ {} }}```, what is the answer? Answer:'

generator = pipeline('text-generation', 
                     model=model, 
                     tokenizer=tokenizer,
                     do_sample=False)


def generate_answers(input_csv_file, output_csv_file):
    with open(input_csv_file, 'r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        data = list(reader)

    # Open the output CSV file in append mode to keep adding rows
    with open(output_csv_file, 'a', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        
        # If the file is empty, write the header
        if outfile.tell() == 0:
            writer.writerow(['Question', 'Generated Answer'])

        for row in data:
            question = row['Question']
            updated_prompt = prompt.format(question)
            result = generator(updated_prompt, max_length=128)
            generated_answer = result[0]['generated_text'].replace(updated_prompt, '').strip()
            writer.writerow([question, generated_answer])

            # Flush the buffer to ensure the data is written to the file immediately
            outfile.flush()

# Example usage
input_csv_file = '/kaggle/input/test-data/updated_Testing_verified_meetings_10.csv'  # Change this to the path of your input CSV file
output_csv_file = 'generated_answers_bloomz1b1_5.8k.csv'  # Change this to the path where you want to save the output CSV file

while True:
    generate_answers(input_csv_file, output_csv_file)
    # Wait for some time before processing the next batch of questions
    time.sleep(300)  # Wait for 5 minutes before processing the next batch
